In [3]:
!pip install yfinance
import yfinance as yf
import pandas as pd
import numpy as np

# Define the list of stock symbols
stocks = ["MSFT", "AAPL", "NVDA", "AMZN", "GOOG", "META", "TSLA"]

# Define the start and end dates
start_date = "2013-01-01"
end_date = "2023-12-31"

# Create an empty DataFrame to store the stock data
stock_data = pd.DataFrame()

# Extract historical price data for each stock and store it in the DataFrame
for symbol in stocks:
    data = yf.download(symbol, start=start_date, end=end_date)
    data = data.dropna()
    stock_data[symbol] = data["Adj Close"]

# Define Bollinger Bands parameters
period = 20
deviations_1 = 1
deviations_2 = 2

# Define trading strategy function
def trading_strategy(price, A1, B1, B2, A2):
    if price < A1 and price > B1:
        return "Buy"
    elif price < B2 and price > A2:
        return "Sell"
    else:
        return "Neutral"

# Define initial capital and minimum transaction size
initial_capital = 10000
min_transaction_size = 1

# Initialize variables to track portfolio and transaction history
portfolio_value = initial_capital
transaction_history = []

# Implement Double Bollinger Band indicators and backtesting loop
for index, row in stock_data.iterrows():
    for symbol in stocks:
        price = row[symbol]
        # Calculate rolling mean and standard deviation
        rolling_mean = stock_data[symbol].rolling(window=period).mean()
        rolling_std = stock_data[symbol].rolling(window=period).std()

        # Calculate upper and lower bands for first Bollinger Bands
        A1 = rolling_mean.loc[index] + (deviations_2 * rolling_std.loc[index]) #Upper band 1
        A2 = rolling_mean.loc[index] - (deviations_2 * rolling_std.loc[index]) #Lower band 1

        # Calculate upper and lower bands for second Bollinger Bands
        B1 = rolling_mean.loc[index] + (deviations_1 * rolling_std.loc[index]) #Upper band 2
        B2 = rolling_mean.loc[index] - (deviations_1 * rolling_std.loc[index]) #Lower band 2

        # Apply trading strategy
        action = trading_strategy(price, A1, B1, B2, A2)
        
        # Execute buy or sell orders based on the trading strategy
        if action == "Buy" and portfolio_value >= price * min_transaction_size:
            num_shares = min(int(portfolio_value / price), min_transaction_size)
            transaction_cost = num_shares * price
            portfolio_value -= transaction_cost
            transaction_history.append((index, symbol, "Buy", num_shares, price))
            # Adjust portfolio value by adding the value of newly bought stocks
            portfolio_value += num_shares * price
        elif action == "Sell":
            # Assuming we sell all shares of the stock
            num_shares = min_transaction_size
            transaction_cost = num_shares * price
            portfolio_value += transaction_cost
            transaction_history.append((index, symbol, "Sell", num_shares, price))
            # Adjust portfolio value by subtracting the value of newly sold stocks
            portfolio_value -= num_shares * price

# Calculate the final capital
final_capital = portfolio_value
for _, _, action, num_shares, price in transaction_history:
    if action == "Buy":
        final_capital += num_shares * price
    elif action == "Sell":
        final_capital -= num_shares * price

# Convert transaction history to DataFrame
transaction_df = pd.DataFrame(transaction_history, columns=["Date", "Symbol", "Action", "Quantity", "Price"])

# Calculate Total Return
total_return = (final_capital - initial_capital) / initial_capital * 100

# Calculate Annual Return
years = (pd.to_datetime(end_date) - pd.to_datetime(start_date)).days / 365
annual_return = ((1 + total_return) ** (1 / years) - 1) * 100

# Calculate Daily Returns
daily_returns = transaction_df.pivot(index='Date', columns='Symbol', values='Price').dropna()
daily_returns = transaction_df.pivot(index='Date', columns='Symbol', values='Price').pct_change(fill_method=None)

# Calculate Annual Volatility
annual_volatility = daily_returns.std().mean() * np.sqrt(252) *100

# Calculate Sharpe Ratio
risk_free_rate = 0.02  # Assuming 2% annual risk-free rate
sharpe_ratio = (annual_return - risk_free_rate) / annual_volatility

# Calculate Sortino Ratio
downside_returns = daily_returns[daily_returns < 0]
downside_volatility = downside_returns.std().mean() * np.sqrt(252)
sortino_ratio = (annual_return - risk_free_rate) / downside_volatility

# Calculate Maximum Drawdown
cumulative_returns = (1 + daily_returns).cumprod()
peak = cumulative_returns.expanding(min_periods=1).max()
drawdown = (cumulative_returns - peak) / peak
max_drawdown = drawdown.min().min() * 100

#Print the results
print("Portfolio Value:", final_capital)
print ("Transaction history:",transaction_df)
print("Total Return:", round (total_return,2), "%")
print("Annual Return:", round(annual_return,2), "%")
print("Annual Volatility:", round(annual_volatility,2), "%")
print("Sharpe Ratio:", round(sharpe_ratio,2))
print("Sortino Ratio:", round(sortino_ratio,2))
print("Maximum Drawdown:", round(max_drawdown,2), "%")

[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed


[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed


Portfolio Value: 314119.3866188526
Transaction history:            Date Symbol Action  Quantity       Price
0    2013-01-30   MSFT    Buy         1   22.720488
1    2013-01-30   AAPL   Sell         1   13.935232
2    2013-01-30   GOOG    Buy         1   18.775375
3    2013-01-30   TSLA    Buy         1    2.501333
4    2013-01-31   AAPL   Sell         1   13.894361
...         ...    ...    ...       ...         ...
8341 2023-12-28   GOOG    Buy         1  141.279999
8342 2023-12-28   META    Buy         1  357.940216
8343 2023-12-29   MSFT    Buy         1  375.345886
8344 2023-12-29   NVDA    Buy         1  495.196777
8345 2023-12-29   META    Buy         1  353.584839

[8346 rows x 5 columns]
Total Return: 3041.19 %
Annual Return: 107.29 %
Annual Volatility: 24.4 %
Sharpe Ratio: 4.4
Sortino Ratio: 550.53
Maximum Drawdown: -45.68 %
